In [ ]:
# Install all required libraries
!pip install unbabel-comet
!pip install -q transformers accelerate evaluate bert_score rouge_score sacremoses nltk psutil pandas sacrebleu sentencepiece

from huggingface_hub import login

# Paste your token here
login(token='hugging face token goes here')

# Download NLTK data needed for BLEU and ROUGE
import nltk
nltk.download('punkt')

# Step 4: Imports
import time
import torch
import psutil
import logging
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
import evaluate

logging.getLogger("transformers").setLevel(logging.ERROR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 5: Paragraph dataset
paragraph_data = [
    {"lang": "ar", "name": "Arabic", "src": "أنا أتقدم للحصول على تأشيرة سياحية لزيارة الولايات المتحدة. أنوي البقاء لمدة أسبوعين وزيارة معالم مشهورة مثل تمثال الحرية وساحة التايمز. سأعود إلى بلدي بعد انتهاء عطلتي.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "es", "name": "Spanish", "src": "Estoy solicitando una visa de turista para visitar los Estados Unidos. Tengo la intención de quedarme dos semanas y visitar lugares famosos como la Estatua de la Libertad y Times Square. Regresaré a mi país después de mis vacaciones.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "fr", "name": "French", "src": "Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ru", "name": "Russian", "src": "Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ro", "name": "Romanian", "src": "Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
]

# Step 6: Load model and tokenizer
model_name = "facebook/m2m100_1.2B"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

# Step 7: Load evaluation modules
bleu_mod      = evaluate.load("bleu")
bertscore_mod = evaluate.load("bertscore")
meteor_mod    = evaluate.load("meteor")
rouge_mod     = evaluate.load("rouge")
ter_mod       = evaluate.load("ter")
comet_mod     = evaluate.load("comet")

# Step 8: System monitor helper
def measure_system():
    proc = psutil.Process()
    return proc.memory_info().rss / 1024**2, psutil.cpu_percent(interval=None)

# Step 9: Translate with M2M100
def translate_m2m100(text: str, src_lang: str, tgt_lang: str = "en"):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt").to(device)
    start = time.perf_counter()
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang)
    )
    mid = time.perf_counter()
    pred = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    end = time.perf_counter()
    return pred, mid - start, end - start, len(tokenizer.tokenize(text))

# Step 10: Evaluate all
for item in paragraph_data:
    src_lang, name, src, ref = item["lang"], item["name"], item["src"], item["ref"]
    print(f"\n=== {name} → English ===")

    ram_before, cpu_before = measure_system()
    pred, t_first, t_total, token_count = translate_m2m100(src, src_lang)
    ram_after, cpu_after = measure_system()

    # Accuracy metrics
    bleu_score   = bleu_mod.compute(predictions=[pred], references=[[ref]])["bleu"]
    bert_res     = bertscore_mod.compute(predictions=[pred], references=[ref], lang="en")
    meteor_score = meteor_mod.compute(predictions=[pred], references=[[ref]])["meteor"]
    rouge_score  = rouge_mod.compute(predictions=[pred], references=[ref])["rougeL"]
    ter_score    = ter_mod.compute(predictions=[pred], references=[[ref]])["score"]
    comet_res    = comet_mod.compute(sources=[src], predictions=[pred], references=[ref])
    comet_score  = next(v for v in comet_res.values() if isinstance(v, (int, float)))

    # Output
    print(f"Source    : {src}")
    print(f"Reference : {ref}")
    print(f"Predicted : {pred}\n")
    print(f"BLEU      : {bleu_score:.4f}")
    print(f"BERTScore : {bert_res['f1'][0]:.4f}")
    print(f"METEOR    : {meteor_score:.4f}")
    print(f"ROUGE-L   : {rouge_score:.4f}")
    print(f"TER       : {ter_score:.4f}")
    print(f"COMET     : {comet_score:.4f}\n")
    print(f"Time to 1st tok : {t_first:.4f}s")
    print(f"Total inf time  : {t_total:.4f}s")
    print(f"Tokens/sec      : {token_count / t_total:.2f}")
    if device.type == "cuda":
        print(f"VRAM used       : {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
    print(f"RAM   : {ram_after - ram_before:.2f} MB")
    print(f"CPU   : {cpu_after - cpu_before:.2f} %")
    print(f"Model size : {sum(p.numel() for p in model.parameters()) * 4 / 1024 ** 2:.2f} MB")
    print("=" * 60)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Source    : Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit famous sites such as the Statue of Liberty and Times Square.

BLEU      : 0.6173
BERTScore : 0.9763
METEOR    : 0.6848
ROUGE-L   : 0.7671
TER       : 34.1463
COMET     : 0.8791

Time to 1st tok : 342.6946s
Total inf time  : 342.7235s
Tokens/sec      : 0.18
RAM   : 221.16 MB
CPU   : -3.30 %
Model size : 4728.20 MB

=== Russian → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I am applying for a tourist visa to travel to the United States. I am going to stay for two weeks and visit famous attractions such as the Statue of Liberty and Times Square.

BLEU      : 0.5045
BERTScore : 0.9682
METEOR    : 0.6575
ROUGE-L   : 0.6933
TER       : 41.4634
COMET     : 0.8712

Time to 1st tok : 344.9406s
Total inf time  : 344.9660s
Tokens/sec      : 0.19
RAM   : -469.31 MB
CPU   : -6.30 %
Model size : 4728.20 MB

=== Romanian → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks such as the Statue of Liberty and Times Square. I will return to my country after the holiday.

BLEU      : 0.8200
BERTScore : 0.9902
METEOR    : 0.9055
ROUGE-L   : 0.9024
TER       : 12.1951
COMET     : 0.9271

Time to 1st tok : 445.2330s
Total inf time  : 445.2626s
Tokens/sec      : 0.13
RAM   : 350.53 MB
CPU   : -10.00 %
Model size : 4728.20 MB
